# Presto Demo
Presto is a fast product recommendation algorithm that can process millions of reviews in milliseconds, providing recommendations similar to the 'users who liked this product also liked' recommendations on an e-commerce site.

This notebook simulates how Presto would work on a production website. Just enter a category (Books, Albums, Artists, or Authors) and a search term to receive instant product recommendations based on millions of user reviews.

For more details on how Presto works, see the numbered notebooks in the presto folder.

In [7]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%autoreload 
%run presto/demo_implementation.ipynb
conn = q.connect('presto/data/products.sql')


Path data/products.sql does not exist in /Users/rolf/Desktop/GA/presto


OperationalError: unable to open database file

OperationalError: unable to open database file

In [9]:
# Search input. These are intended as user-facing inputs.
type = 'Book' # possibilities: Book, Album, Artist, Author
search = "The Hobbit"
exact_match = False

# recommender settings
# feel free to experiment with these to see how results change.
max_search_results = 10 # Relevant in case of ambiguous search terms with multiple hits. Presto will resolve ambiguity by selecting the most-reviewd product
max_recommendations = 12 # How many of Presto's related products to display to the usser
recommender_user_pool_size = 100 # For large numbers of user reviews, consider only this many users for recommendations. Filters by users who leave the most reviews.
recommender_product_pool_size = 100 # For large numbers of user reviews, consider only this many products for recommendations. Filters by number of reviews

log_verbosity = 1 # Increase to see internal details
include_subtitles = True

###################################
# Recommendations
##################################
results = output_recommendations(type, search, exact_match, conn = conn, include_subtitles = include_subtitles)

NameError: name 'conn' is not defined

In [3]:
# Explore the internals of the results in more detail if needed:
product = results['result']
products = results['results']
recommendations = results['recommendations']
product

NameError: name 'results' is not defined

In [4]:
products

NameError: name 'products' is not defined

In [ ]:
recommendations

In [ ]:
# Don' forget to clean up
conn.close()